<a href="https://colab.research.google.com/github/CyberSnorlax/Anticipatory-Actions-for-Climate-Risk/blob/Climate-and-Vegetation-Monitoring-for-Chad-(July%E2%80%93September-2022)/Chad_Drought_Assessment_(1994_2023).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!earthengine-authenticate

/bin/bash: line 1: earthengine-authenticate: command not found


In [ ]:
pip install earthengine-api

In [ ]:
# create sections for each variable
# do for 3 months, July, Aug, Sept, calculate avg value for precip, soil moisture, vegestation
# display on map
# do it for admin level one - country
# admin level two - state and provinces
# we want data to be in the form of a table

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Initialize Earth Engine
ee.Authenticate()

ee.Initialize(project='data-centaur-452217-j1')

In [ ]:
# Load Administrative Boundaries
country = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Chad'))
admin1 = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM0_NAME', 'Chad'))

In [ ]:
# Define the 3-month period (July to September)
start_month = 7  # July
end_month = 9    # September
year = 2022      # Using 2022 for complete data availability

# Load Datasets
srtm = ee.Image('USGS/SRTMGL1_003')              # Elevation
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')  # Precipitation
smap = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007')   # Soil Moisture
modis_vi = ee.ImageCollection('MODIS/006/MOD13A2')    # Vegetation Indices

/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13A2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
# Section 1: Precipitation
def get_precipitation(geometry, year):
    start_date = f'{year}-{start_month:02d}-01'
    end_date = f'{year}-{end_month:02d}-30'
    precip = (chirps.filterDate(start_date, end_date)
              .sum()
              .reduceRegions(reducer=ee.Reducer.mean(), collection=geometry, scale=5000))
    return precip

In [ ]:
# Section 2: Soil Moisture
def get_soil_moisture(geometry, year):
    start_date = f'{year}-{start_month:02d}-01'
    end_date = f'{year}-{end_month:02d}-30'
    soil_moisture = (smap.filterDate(start_date, end_date)
                     .select('sm_surface')
                     .mean()
                     .reduceRegions(reducer=ee.Reducer.mean(), collection=geometry, scale=10000))
    return soil_moisture

In [ ]:
# Section 3: Vegetation Indices (NDVI and EVI)
def get_vegetation(geometry, year):
    start_date = f'{year}-{start_month:02d}-01'
    end_date = f'{year}-{end_month:02d}-30'
    vi = (modis_vi.filterDate(start_date, end_date)
          .select(['NDVI', 'EVI'])
          .mean()
          .reduceRegions(reducer=ee.Reducer.mean(), collection=geometry, scale=1000))
    return vi

In [ ]:
# Calculate data for Admin Level 0 (Country)
precip_country = get_precipitation(country, year).getInfo()
sm_country = get_soil_moisture(country, year).getInfo()
vi_country = get_vegetation(country, year).getInfo()

In [ ]:
# Calculate data for Admin Level 1 (States/Provinces)
precip_admin1 = get_precipitation(admin1, year).getInfo()
sm_admin1 = get_soil_moisture(admin1, year).getInfo()
vi_admin1 = get_vegetation(admin1, year).getInfo()

In [ ]:
# Convert to DataFrames
def features_to_df(features, columns):
    data = []
    for feature in features['features']:
        props = feature['properties']
        row = {col: props.get(col) for col in columns}
        row['name'] = props.get('ADM1_NAME') if 'ADM1_NAME' in props else 'Chad'
        data.append(row)
    return pd.DataFrame(data)

In [ ]:
# Country-level table
country_df = pd.concat([
    features_to_df(precip_country, ['mean']).rename(columns={'mean': 'precipitation'}),
    features_to_df(sm_country, ['mean']).rename(columns={'mean': 'soil_moisture'}),
    features_to_df(vi_country, ['NDVI', 'EVI'])
], axis=1).drop_duplicates(subset=['name'])

# Admin1-level table
admin1_df = pd.concat([
    features_to_df(precip_admin1, ['mean']).rename(columns={'mean': 'precipitation'}),
    features_to_df(sm_admin1, ['mean']).rename(columns={'mean': 'soil_moisture'}),
    features_to_df(vi_admin1, ['NDVI', 'EVI'])
], axis=1).drop_duplicates(subset=['name'])

# Adjust NDVI and EVI scales (MODIS NDVI/EVI are scaled by 0.0001)
admin1_df['NDVI'] = admin1_df['NDVI'] * 0.0001
admin1_df['EVI'] = admin1_df['EVI'] * 0.0001
country_df['NDVI'] = country_df['NDVI'] * 0.0001
country_df['EVI'] = country_df['EVI'] * 0.0001

In [ ]:
# Display Tables
print("Country-Level Data (Chad, July-Sept 2022):")
print(country_df[['name', 'precipitation', 'soil_moisture', 'NDVI', 'EVI']])
print("\nAdmin1-Level Data (Chad States/Provinces, July-Sept 2022):")
print(admin1_df[['name', 'precipitation', 'soil_moisture', 'NDVI', 'EVI']].head())


Country-Level Data (Chad, July-Sept 2022):
   name  name  name  precipitation  soil_moisture      NDVI      EVI
0  Chad  Chad  Chad     306.204976       0.147319  0.312853  0.21665

Admin1-Level Data (Chad States/Provinces, July-Sept 2022):
          name         name         name  precipitation  soil_moisture  \
0    Batha Est    Batha Est    Batha Est     432.948250       0.175902   
1  Batha Ouest  Batha Ouest  Batha Ouest     357.305065       0.161695   
2       Borkou       Borkou       Borkou      19.171095       0.048794   
3       Ennedi       Ennedi       Ennedi      65.326570       0.065940   
4      Tibesti      Tibesti      Tibesti       5.926355       0.049166   

       NDVI       EVI  
0  0.439720  0.320014  
1  0.405320  0.301217  
2  0.095338  0.073333  
3  0.111239  0.079818  
4  0.089887  0.066006  


In [ ]:
# Create Map
Map = geemap.Map()

In [ ]:
# Add Layers for Admin1
precip_image = (chirps.filterDate(f'{year}-07-01', f'{year}-09-30').sum().clip(admin1))
sm_image = (smap.filterDate(f'{year}-07-01', f'{year}-09-30').select('sm_surface').mean().clip(admin1))
vi_image = (modis_vi.filterDate(f'{year}-07-01', f'{year}-09-30').select('NDVI').mean().clip(admin1))

Map.addLayer(admin1, {}, 'Admin1 Boundaries')
Map.addLayer(precip_image, {'min': 0, 'max': 500, 'palette': ['blue', 'green', 'yellow']}, 'Precipitation')
Map.addLayer(sm_image, {'min': 0, 'max': 0.3, 'palette': ['red', 'yellow', 'green']}, 'Soil Moisture')
Map.addLayer(vi_image, {'min': 0, 'max': 10000, 'palette': ['brown', 'yellow', 'green']}, 'NDVI')

# Center the map
Map.centerObject(country, 5)

# Display the map
Map

Map(center=[15.282206186229162, 18.65135847264551], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Additional Info from FAO GIEWS Country Brief
print("\nFAO GIEWS Country Brief Highlights (Chad):")
print("- Main agricultural season: June to October (millet, sorghum)")
print("- Precipitation data aligns with rainy season (July-Sept avg used here)")
print("- Vegetation indices (NDVI/EVI) reflect crop health during growing season")


FAO GIEWS Country Brief Highlights (Chad):
- Main agricultural season: June to October (millet, sorghum)
- Precipitation data aligns with rainy season (July-Sept avg used here)
- Vegetation indices (NDVI/EVI) reflect crop health during growing season
